## Imports

In [8]:
from bs4 import BeautifulSoup
import urllib
import os

In [2]:
import spotipy
import spotipy.util as util

## Parse HTML

In [3]:
def get_beaut_soup(url):
    
    with urllib.request.urlopen(url) as response:
        response = response.read()
    page = BeautifulSoup(response, 'html.parser')
    
    return page

In [4]:
def clean_headliner(headliner):
    if "(" in headliner:
        return headliner.split(" (")[0]
    else:
        return headliner

In [5]:
def parse_r5(url="https://www.r5productions.com/"):
    
    headliners = []
    page = get_beaut_soup(url)
    headliners_html = page.find_all("h1")
    
    headliners = [clean_headliner(headliner.text) for headliner in headliners_html if '\n' not in headliner.text]
    headliners = list(set(headliners))
    return headliners

In [6]:
artists = parse_r5()

## Spotify Authentication

In [10]:
client_id = os.environ['SPOTIFY_CLIENT_ID']
client_secret = os.environ['SPOTIFY_CLIENT_SECRET']

In [43]:
username="" #Insert Username
scope = "playlist-modify-private"

In [12]:
token = util.prompt_for_user_token(username,
                                       scope,
                                       client_id=client_id,
                                       client_secret=client_secret,
                                       redirect_uri="http://localhost:8888")

In [13]:
service = spotipy.Spotify(auth=token)

## Get Tracks

In [14]:
def get_spotify_uri(name, service):
    results = service.search(q="artist:" + name, type="artist")
    uri = ""
    try:
        uri = results["artists"]["items"][0]["uri"]
    except:
        pass
    return uri

In [15]:
def get_top_tracks(uri, service, number=3):
    results = service.artist_top_tracks(uri)
    tracks = [track['id'] for track in results["tracks"][:number]]
    return tracks

## Create Playlist

In [16]:
def make_playlist_from_tracks(username, playlist_name, tracks, service, playlist_description=""):
    playlist = service.user_playlist_create(username, playlist_name, False)
    
    playlist_id = playlist["id"]
    
    service.user_playlist_add_tracks(username, playlist_id, tracks)
    
    return

In [17]:
def make_playlist(service, username, artists, date):
    uris = [get_spotify_uri(artist, service) for artist in artists]
    tracks = [track for uri in uris for track in get_top_tracks(uri, service)]
    
    playlist_name = "R5 Playlist " + date
    make_playlist_from_tracks(username, playlist_name, tracks, service)
    
    return

In [18]:
make_playlist(service, username, artists, "Feb 2018")